In [210]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import statistics
from collections import Counter
import pyarrow as pa
# import pyarrow.parquet as pq
import os
from collections import OrderedDict
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
sns.set_style("whitegrid")

In [226]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

In [227]:
taskDF = pd.read_parquet(filePath)
taskDF['ParticipantNum'] = taskDF["ParticipantPrivateID"]
taskDF = taskDF.astype({"ParticipantPrivateID": str})
filePath = os.path.join(tempTopDir, 'GP_Main_Short.parquet')
slimDF = pd.read_parquet(filePath)
print(len(slimDF))

249


In [228]:
filePath = os.path.join(tempTopDir, 'GP_Consent_Long.parquet')
consentDF_L = pd.read_parquet(filePath).infer_objects()
consentDF_L = consentDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_L = consentDF_L[consentDF_L['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_Consent_Short.parquet')
consentDF_S = pd.read_parquet(filePath).infer_objects()
consentDF_S = consentDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
consentDF_S = consentDF_S[consentDF_S['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_Demographics_Long.parquet')
demoDF_L = pd.read_parquet(filePath).infer_objects()
demoDF_L = demoDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_L = demoDF_L[demoDF_L['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_Demographics_Short.parquet')
demoDF_S = pd.read_parquet(filePath).infer_objects()
demoDF_S = demoDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
demoDF_S = demoDF_S[demoDF_S['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_AI_Long.parquet')
aiDF_L = pd.read_parquet(filePath).infer_objects()
aiDF_L = aiDF_L.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_L = aiDF_L[aiDF_L['ParticipantPrivateID'].notna()]

filePath = os.path.join(tempTopDir, 'GP_AI_Short.parquet')
aiDF_S = pd.read_parquet(filePath).infer_objects()
aiDF_S = aiDF_S.astype({"ParticipantPrivateID": str})
# drop rows where NAN in participant private id
aiDF_S = aiDF_S[aiDF_S['ParticipantPrivateID'].notna()]

In [229]:
print(len(slimDF.ParticipantPublicID.unique()))

# drop particpants who have not been accepted (rejected or timed out)
# options: approved, awaiting review,rejected, returned, timed-out
print(slimDF.ExperimentVersion.unique())
print(slimDF.status.unique())
# get names of indexes for which to drop
pID_removed1 = slimDF[(slimDF['status']=='REJECTED') | 
                      (slimDF['status']=='RETURNED')].ParticipantPublicID.unique()
pID_removed2 = slimDF[(slimDF['status']=='REJECTED')].ParticipantPublicID.unique()
index_names = slimDF[(slimDF['status']!='AWAITING REVIEW') & (slimDF['status']!='APPROVED')].index
  
# drop these given row
# indexes from dataFrame

# tmp_slimDF = slimDF[~slimDF['ParticipantPublicID'].isin(pID_removed1)].copy()
print(tmp_slimDF.status.unique())
print(pID_removed1)
    
tmp_demoDF_L = demoDF_L[~demoDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
tmp_demoDF_S = demoDF_S[~demoDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

tmp_consentDF_L = consentDF_L[~consentDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
tmp_consentDF_S = consentDF_S[~consentDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

tmp_aiDF_L = aiDF_L[~aiDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
tmp_aiDF_S = aiDF_S[~aiDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

tmp_taskDF = taskDF[~taskDF['ParticipantPublicID'].isin(pID_removed1)].copy()
tmp_slimDF = slimDF[slimDF['ParticipantPublicID'].isin(tmp_taskDF.ParticipantPublicID.unique())].copy()
# print(tmp_taskDF[tmp_taskDF['ParticipantPublicID']==pID_removed1[-3]])

print(len(tmp_slimDF.ParticipantPublicID.unique()))
print(len(tmp_taskDF.ParticipantPublicID.unique()))

245
[60. 62. 64. 65. 67. 70. 72. 73. 76. 77.]
['APPROVED' 'RETURNED' 'REJECTED']
['APPROVED']
['5f5b7c0c291dc62b000bb637' '5f57a42b46df8927414990fc'
 '601f23aa1dc6d94683bde1ff' '5fcb5f768c4bb789f899288f'
 '60342dbfcf15a5092cda30dc' '5bce1cf2ac6b660001908ab8'
 '5f102f0212ea4f000a8ac9da' '600ac30a0e8a51025de63b95'
 '5f40d6c251a8cd21d01b6f40' '6034f9fe12d7b21679665d41'
 '5eccd97f6ce039000b2a61f2' '5ca14d419ca7f60017258fc6'
 '5f8dcac25e91d312a21a625f']
232
232


In [230]:
likertDF = tmp_taskDF.copy()

In [231]:
## BREAK THE TEXT INTO BITS TO NAME THE LIKERTS ##

likertDF['likertName'] = 0

tmp = likertDF.query('Likert_Num!=0 & ZoneType == "response_button_text"')

new = tmp['Text1'].str.split(" ", n = 10, expand = True)

# now, make a new column and then update in main DF

tmp['likertName'] = new[0]+"_"+new[1]+"_"+new[2]+"_"+new[3]+"_"+new[4]+"_"+new[5]+"_"+new[6]
ls = tmp['likertName'].unique().tolist()
for i in ls:
    print(i)
tmp['likertName'] = tmp['likertName'].str.replace('<h3>The_charts_were_helpful_in_answering_the', \
                                                  'charts_helpful',regex=True)

tmp['likertName'] = tmp['likertName'].str.replace('<h3>The_scenarios_shown_were_helpful_in_answering', \
                                                  'scenarios_helpful',regex=True)

tmp['likertName'] = tmp['likertName'].str.replace('<h3>The_blurring_of_the_video_was_helpful', \
                                                  'blurring_helpful',regex=True)

tmp['likertName'] = tmp['likertName'].str.replace('<h3>The_charts_contained_sufficient_detail_for_me', \
                                                  'charts_helpful',regex=True)

tmp['likertName'] = tmp['likertName'].str.replace('_<h3>The_charts_contained_irrelevant_or_distracting', \
                                                  'irrelevant_distracting',regex=True)

tmp['likertName'] = tmp['likertName'].str.replace('<h3>The_information_shown_was_sufficient_for_understanding', \
                                                  'info_sufficient',regex=True)

tmp['likertName'] = tmp['likertName'].str.replace( \
                                          '<h3>The_charts_contained_irrelevant_or_distracting_details.</h3>', \
                                                  'irrelevant_distracting',regex=True)

tmp['likertName'] = tmp['likertName'].str.replace( \
                                          '<h3>The_scenarios_contained_irrelevant_or_distracting_details.</h3>', \
                                                  'irrelevant_distracting',regex=True)

tmp['likertName'] = np.where(tmp['AgentNum'].astype(int) == 1, tmp['likertName'] + "_1",
                             np.where(tmp['AgentNum'].astype(int) == 2, tmp['likertName'] + "_2",
                                      tmp['likertName'] + "_0"))
                             

print(tmp['likertName'].unique())

likertDF['likertName'].update(tmp['likertName'])

print(likertDF.query('Likert_Num!=0')['likertName'])

print(likertDF.query('Likert_Num==0')['likertName'])

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying t

<h3>The_charts_were_helpful_in_answering_the
<h3>The_scenarios_shown_were_helpful_in_answering
<h3>The_blurring_of_the_video_was_helpful
<h3>The_information_shown_was_sufficient_for_understanding
<h3>The_charts_contained_sufficient_detail_for_me
_<h3>The_charts_contained_irrelevant_or_distracting
<h3>The_charts_contained_irrelevant_or_distracting_details.</h3>
<h3>The_scenarios_contained_irrelevant_or_distracting_details.</h3>
['charts_helpful_1' 'scenarios_helpful_1' 'blurring_helpful_1'
 'info_sufficient_1' 'charts_helpful_2' 'scenarios_helpful_2'
 'blurring_helpful_2' 'info_sufficient_2' 'irrelevant_distracting_1'
 'irrelevant_distracting_2']
index
57          charts_helpful_1
58       scenarios_helpful_1
59        blurring_helpful_1
60         info_sufficient_1
86          charts_helpful_2
                ...         
75565     blurring_helpful_1
75570      info_sufficient_1
75635    scenarios_helpful_2
75640     blurring_helpful_2
75645      info_sufficient_2
Name: likertName, Len

In [232]:
#################Pull out the rows we need with just the likerts info#####################
tmp1 = likertDF[['Response', 'likertName', 'ParticipantPublicID','ZoneType','display']].query('likertName!=0 & ZoneType == "response_button_text"')
tmp1['Response'] = pd.Categorical(tmp1['Response'])
tmp1 = tmp1.dropna()
# print(len(tmp1[tmp1['ParticipantPublicID']==pID].query('display=="likerts"')))



In [233]:
# first filter down to only irrelevant:
tmp2 = tmp1.query('likertName=="irrelevant_distracting_1" | likertName=="irrelevant_distracting_2"')
codes = {'Very False':'Very True','Somewhat False':'Somewhat True', \
         'Neutral':'Neutral','Somewhat True':'Somewhat False','Very True':'Very False'}

# Now use map on sub-df to swap negative q sentiments
tmp2['Response1'] = tmp2['Response'].map(codes)


/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [234]:
# Add the swapped sentiments back to the original DF
tmp1['Response'].update(tmp2['Response1'])
# print(len(tmp1[tmp1['ParticipantPublicID']==pID].query('display=="likerts"')))

In [235]:
# now use mapping to make numeric column
codes = {'Very False':1,'Somewhat False':2, 'Neutral':3,'Somewhat True':4,'Very True':5}
tmp1['NumericResponse'] = tmp1['Response'].map(codes)
# print(len(tmp1[tmp1['ParticipantPublicID']==pID].query('display=="likerts"')))

In [236]:
# And update main DF with information
likertDF['Response'].update(tmp1['Response'])
likertDF['NumericResponse']=0
likertDF['NumericResponse'].update(tmp1['NumericResponse'])
# likertDF = pd.merge(likertDF,tmp1[['NumericResponse', 'ParticipantPublicID','ZoneType','display']], on=['ZoneType','ParticipantPublicID','display'],how="left")
# print(len(likertDF[likertDF['ParticipantPublicID']==pID].query('display=="likerts"')))


In [237]:
# Copy these so we can start over from here if needed

likertSlim = tmp_slimDF.copy()
likertDF2 = likertDF.copy()

print(len(likertSlim))
print(len(likertDF2))

232
71763


In [238]:

# Need to add to slimDF... User's numeric response, and user's normalized response, for each.
print(tmp_taskDF.ZoneType.unique())

df_LikertPivot = likertDF2.query('likertName!=0 & ZoneType == "response_button_text"')[['NumericResponse', 'likertName','ParticipantPublicID']]
df_LikertPivot.drop_duplicates(['likertName','ParticipantPublicID'],inplace=True)
print(df_LikertPivot)

df_LikertPivot.set_index(['ParticipantPublicID'], append=True)
tmp2 =  df_LikertPivot[['NumericResponse', 'likertName','ParticipantPublicID']].pivot(index='ParticipantPublicID', columns=['likertName'], values='NumericResponse')
tmp2 = tmp2.reset_index()

print(tmp2.head(5))
print(likertSlim.columns.tolist())
print(tmp2.columns.tolist())

likertSlim = pd.merge(likertSlim,tmp2[['ParticipantPublicID', 'blurring_helpful_1', \
      'blurring_helpful_2', 'charts_helpful_1', 'charts_helpful_2', 'info_sufficient_1', 'info_sufficient_2', 'irrelevant_distracting_1', \
      'irrelevant_distracting_2', 'scenarios_helpful_1', 'scenarios_helpful_2']], \
                on='ParticipantPublicID',how="left")

# print(likertSlim[likertSlim['ParticipantPublicID']==pID][['ParticipantPublicID', 'blurring_helpful_1', \
#       'blurring_helpful_2', 'charts_helpful_1', 'charts_helpful_2', 'info_sufficient_1', 'info_sufficient_2', 'irrelevant_distracting_1', \
#       'irrelevant_distracting_2', 'scenarios_helpful_1', 'scenarios_helpful_2']])


[None 'content_video' 'continue_button' 'response_button_text'
 'response_text_area' 'response_rating_scale_likert_active'
 'response_rating_scale_likert']
       NumericResponse           likertName       ParticipantPublicID
index                                                                
57                 4.0     charts_helpful_1  5fc94c64c8a3e03ded162dd4
58                 5.0  scenarios_helpful_1  5fc94c64c8a3e03ded162dd4
59                 5.0   blurring_helpful_1  5fc94c64c8a3e03ded162dd4
60                 3.0    info_sufficient_1  5fc94c64c8a3e03ded162dd4
86                 4.0     charts_helpful_2  5fc94c64c8a3e03ded162dd4
...                ...                  ...                       ...
75565              3.0   blurring_helpful_1  5ed57ec8141d24118731f1bc
75570              4.0    info_sufficient_1  5ed57ec8141d24118731f1bc
75635              4.0  scenarios_helpful_2  5ed57ec8141d24118731f1bc
75640              4.0   blurring_helpful_2  5ed57ec8141d24118731f1bc
7564

In [239]:
# Get mode of likert anwsers on 1, 2, and overall

tmp = likertSlim.copy()
print(len(likertSlim))
tmp1 = tmp[['blurring_helpful_1', \
      'blurring_helpful_2', 'charts_helpful_1', 'charts_helpful_2', 'info_sufficient_1', 'info_sufficient_2', 'irrelevant_distracting_1', \
      'irrelevant_distracting_2', 'scenarios_helpful_1', 'scenarios_helpful_2']]
print(tmp1)
tmp1['likertMode'] = np.where(tmp1.mode(axis=1)[2].notnull(),tmp1.mode(axis=1)[1],tmp1.mode(axis=1)[0])
# tmp1['likertMode'] = tmp1.mode(axis=1)
print(tmp1['likertMode'])

tmp2 = tmp1[['blurring_helpful_1', 'charts_helpful_1', 'info_sufficient_1', 'irrelevant_distracting_1', \
      'scenarios_helpful_1']]

tmp1['likert1Mode'] = np.where(tmp2.mode(axis=1)[2].notnull(),tmp2.mode(axis=1)[1],tmp2.mode(axis=1)[0])
# tmp1['likert1Mode'] = tmp2.mode(axis=1)
print(tmp1['likert1Mode'])

tmp2 = tmp1[['blurring_helpful_2', 'charts_helpful_2', 'info_sufficient_2', 'irrelevant_distracting_2', \
      'scenarios_helpful_2']]

tmp1['likert2Mode'] = np.where(tmp2.mode(axis=1)[2].notnull(),tmp2.mode(axis=1)[1],tmp2.mode(axis=1)[0])
# tmp1['likert2Mode'] = tmp2.mode(axis=1)
print(tmp1['likert2Mode'])

likertSlim['likertMode'] = tmp1['likertMode']
likertSlim['likert1Mode'] = tmp1['likert1Mode']
likertSlim['likert2Mode'] = tmp1['likert2Mode']

print(len(likertSlim))

232
     blurring_helpful_1  blurring_helpful_2  charts_helpful_1  \
0                   5.0                 5.0               4.0   
1                   NaN                 NaN               4.0   
2                   1.0                 4.0               NaN   
3                   1.0                 1.0               3.0   
4                   2.0                 3.0               1.0   
..                  ...                 ...               ...   
227                 2.0                 2.0               NaN   
228                 NaN                 NaN               3.0   
229                 4.0                 4.0               4.0   
230                 NaN                 NaN               4.0   
231                 3.0                 4.0               NaN   

     charts_helpful_2  info_sufficient_1  info_sufficient_2  \
0                 4.0                3.0                3.0   
1                 4.0                4.0                4.0   
2                 NaN     

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


0      5.0
1      4.0
2      4.0
3      2.0
4      1.0
      ... 
227    4.0
228    3.0
229    4.0
230    3.0
231    4.0
Name: likertMode, Length: 232, dtype: float64


/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


0      5.0
1      4.0
2      2.0
3      2.0
4      1.0
      ... 
227    5.0
228    4.0
229    4.0
230    4.0
231    3.0
Name: likert1Mode, Length: 232, dtype: float64
0      5.0
1      4.0
2      4.0
3      2.0
4      2.0
      ... 
227    4.0
228    2.0
229    4.0
230    3.0
231    4.0
Name: likert2Mode, Length: 232, dtype: float64
232


/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [240]:
## Make DF for answers to comparisons
df_CompPivot = tmp_taskDF.query('AgentNum==3 & (Question!=0 | CompConf_Num!=0)')[['Response','Correct','Question', \
                                                              'ParticipantPublicID', 'CompConf_Num']]
thisDF = slimDF.copy()
df_CompPivot['Response'] = pd.Categorical(df_CompPivot.Response)
print(df_CompPivot.head(4))

#move confcatvar and compconf up one
df_CompPivot['Response'] = df_CompPivot['Response'].shift(-1)
df_CompPivot['CompConf_Num'] = df_CompPivot['CompConf_Num'].shift(-1)
# df_CompPivot['confCatVar'] = df_CompPivot['confCatVar'].shift(-1)
df_CompPivot = df_CompPivot.query('Question!=0').dropna().rename(columns={'Question': "likertName"}).drop(columns=['CompConf_Num','Correct'])
print(df_CompPivot.Response.unique())

codes = {'Not at All Confident':1,'Not Confident':2, \
         'Neutral':3,'Somewhat Confident':4,'Completely Confident':5}
# Now use map on sub-df to swap negative q sentiments
df_CompPivot['NumericResponse'] = df_CompPivot['Response'].map(codes)
df_CompPivot.drop('Response', axis=1, inplace=True)
print(df_CompPivot.head(7))

                 Response  Correct  Question       ParticipantPublicID  \
index                                                                    
101        Agent Thompson      0.0      17.0  5fc94c64c8a3e03ded162dd4   
102    Somewhat Confident      0.0       0.0  5fc94c64c8a3e03ded162dd4   
107         Agent Jackson      1.0      18.0  5fc94c64c8a3e03ded162dd4   
108               Neutral      0.0       0.0  5fc94c64c8a3e03ded162dd4   

       CompConf_Num  
index                
101             0.0  
102             1.0  
107             0.0  
108             2.0  
['Somewhat Confident', 'Neutral', 'Not Confident', 'Completely Confident', 'Not at All Confident']
Categories (11, object): ['Agent Jackson', 'Agent Jones', 'Agent Smith', 'Agent Thompson', ..., 'Neutral', 'Not Confident', 'Not at All Confident', 'Somewhat Confident']
       likertName       ParticipantPublicID  NumericResponse
index                                                       
101          17.0  5fc94c64c8a3e

In [241]:
# Now we need to get zscores:
newDF = pd.DataFrame()

for p_index, pID in enumerate(df_LikertPivot.ParticipantPublicID.unique()):
    tmp = df_LikertPivot[df_LikertPivot['ParticipantPublicID']==pID][['NumericResponse']]
    tmp['ParticipantPublicID']=pID
    tmp['likertName'] = df_LikertPivot[df_LikertPivot['ParticipantPublicID']==pID]['likertName']
    # Need to add in comparison numeric answers....
    tmp = tmp.append(df_CompPivot[df_CompPivot['ParticipantPublicID']==pID])
#     print(tmp)
    tmp = tmp.dropna()
    
    # need to first check if all the same values, as this breaks zscore
    if (len(np.unique(tmp.NumericResponse))==1):
        print(pID)
        tmp = tmp.rename(columns={'NumericResponse': 'Z_Scores', 'Question':'likertName'})
        tmp['likertName'] = tmp['likertName'].astype(str)+"_Z"
        tmp['Z_Scores'] = 0
#         print("All same")
#         print(tmp)
        newDF = newDF.append(tmp)
    else:
#         print(pID)
        tmp['Z_Scores'] = stats.zscore(tmp['NumericResponse'])
        tmp['ParticipantPublicID']=pID
        tmp['likertName'] = tmp['likertName'].astype(str)+"_Z"
        tmp = tmp.reset_index()
        tmp = tmp.drop('NumericResponse',axis=1)
#         print("Varied")
#         print(tmp)
        newDF = newDF.append(tmp)
#     print("Stacked as we go")
#     print(newDF)
# print("Final product")
print(newDF.columns.tolist())
    # Once calculated, need to make little DF and stack them up.... 

5f653485fa6c900b6548f35c
5ce7046a9390750017b64ac6
5ebbe8e3c1aef8016cc78ca1
604b528b95d6d998726f0bd9
5afdb5993eb22c00019bb339
['index', 'ParticipantPublicID', 'likertName', 'Z_Scores']


In [242]:
print(newDF.likertName.unique())
print(newDF[newDF['ParticipantPublicID']==pID])

['charts_helpful_1_Z' 'scenarios_helpful_1_Z' 'blurring_helpful_1_Z'
 'info_sufficient_1_Z' 'charts_helpful_2_Z' 'scenarios_helpful_2_Z'
 'blurring_helpful_2_Z' 'info_sufficient_2_Z' '17.0_Z' '18.0_Z' '19.0_Z'
 '20.0_Z' 'irrelevant_distracting_1_Z' 'irrelevant_distracting_2_Z']
     index       ParticipantPublicID             likertName  Z_Scores
0  75560.0  5ed57ec8141d24118731f1bc  scenarios_helpful_1_Z -0.904534
1  75565.0  5ed57ec8141d24118731f1bc   blurring_helpful_1_Z -0.904534
2  75570.0  5ed57ec8141d24118731f1bc    info_sufficient_1_Z  0.603023
3  75635.0  5ed57ec8141d24118731f1bc  scenarios_helpful_2_Z  0.603023
4  75640.0  5ed57ec8141d24118731f1bc   blurring_helpful_2_Z  0.603023
5  75645.0  5ed57ec8141d24118731f1bc    info_sufficient_2_Z  0.603023
6  75662.0  5ed57ec8141d24118731f1bc                 17.0_Z  0.603023
7  75700.0  5ed57ec8141d24118731f1bc                 18.0_Z  0.603023
8  75738.0  5ed57ec8141d24118731f1bc                 19.0_Z  0.603023
9  75776.0  5ed57ec81

In [243]:
# instead of below, set both indices to pID and then sort on index, concat, and reset index
testDF = likertSlim.copy()

tmp2 =  newDF[['Z_Scores', 'likertName','ParticipantPublicID']].pivot(index='ParticipantPublicID', columns=['likertName'], values='Z_Scores').rename_axis(None, axis=1)
print(tmp2.loc[pID][['17.0_Z', '18.0_Z', '19.0_Z', '20.0_Z', \
                   'blurring_helpful_1_Z', 'blurring_helpful_2_Z', 'info_sufficient_1_Z', \
                   'info_sufficient_2_Z', 'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z']])

# print(tmp2.iloc[:5])
tmp2.sort_index()
print(tmp2.loc[pID][['17.0_Z', '18.0_Z', '19.0_Z', '20.0_Z', \
                   'blurring_helpful_1_Z', 'blurring_helpful_2_Z', 'info_sufficient_1_Z', \
                   'info_sufficient_2_Z', 'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z']])
testDF.set_index('ParticipantPublicID').sort_index()

testDF = pd.concat([testDF,tmp2], axis=1)

testDF = testDF.reset_index()
print(testDF[testDF['ParticipantPublicID']==pID][['ParticipantPublicID', '17.0_Z', '18.0_Z', '19.0_Z', '20.0_Z', \
                   'blurring_helpful_1_Z', 'blurring_helpful_2_Z', 'info_sufficient_1_Z', \
                   'info_sufficient_2_Z', 'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z']])

17.0_Z                   0.603023
18.0_Z                   0.603023
19.0_Z                   0.603023
20.0_Z                  -2.412091
blurring_helpful_1_Z    -0.904534
blurring_helpful_2_Z     0.603023
info_sufficient_1_Z      0.603023
info_sufficient_2_Z      0.603023
scenarios_helpful_1_Z   -0.904534
scenarios_helpful_2_Z    0.603023
Name: 5ed57ec8141d24118731f1bc, dtype: float64
17.0_Z                   0.603023
18.0_Z                   0.603023
19.0_Z                   0.603023
20.0_Z                  -2.412091
blurring_helpful_1_Z    -0.904534
blurring_helpful_2_Z     0.603023
info_sufficient_1_Z      0.603023
info_sufficient_2_Z      0.603023
scenarios_helpful_1_Z   -0.904534
scenarios_helpful_2_Z    0.603023
Name: 5ed57ec8141d24118731f1bc, dtype: float64
          ParticipantPublicID  17.0_Z  18.0_Z  19.0_Z  20.0_Z  \
231  5ed57ec8141d24118731f1bc     NaN     NaN     NaN     NaN   

     blurring_helpful_1_Z  blurring_helpful_2_Z  info_sufficient_1_Z  \
231                   N

In [244]:

############STOPPED HERE####### 

# Need to get tmp into pivoted format and plop onto end of SlimDF!!
testDF = likertSlim.copy()

tmp2 =  newDF[['Z_Scores', 'likertName','ParticipantPublicID']].pivot(index='ParticipantPublicID', columns=['likertName'], values='Z_Scores')
# tmp2.set_index('ParticipantPublicID')
tmp2 = tmp2.reset_index()
# print(idx)
# idx.rename('ParticipantPublicID')
# tmp2 =  df_LikertPivot.groupby('ParticipantPublicID').sum().stack()
# print(tmp2.head(5))
# print(likertSlim.columns.tolist())
# print(tmp2.columns.tolist())
testDF = pd.merge(testDF,tmp2[['ParticipantPublicID', '17.0_Z', '18.0_Z', '19.0_Z', '20.0_Z', \
                   'blurring_helpful_1_Z', 'blurring_helpful_2_Z', 'info_sufficient_1_Z', \
                   'info_sufficient_2_Z', 'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z',
                              'irrelevant_distracting_1_Z', 'irrelevant_distracting_2_Z', \
                              'charts_helpful_1_Z', 'charts_helpful_2_Z']], \
                    on='ParticipantPublicID',how="outer")

print(testDF[testDF['ParticipantPublicID']==pID][['ParticipantPublicID', '17.0_Z', '18.0_Z', '19.0_Z', '20.0_Z', \
                   'blurring_helpful_1_Z', 'blurring_helpful_2_Z', 'info_sufficient_1_Z', \
                   'info_sufficient_2_Z', 'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z',
                              'irrelevant_distracting_1_Z', 'irrelevant_distracting_2_Z', \
                              'charts_helpful_1_Z', 'charts_helpful_2_Z']])
print(len(testDF))
print(testDF[['ParticipantPublicID', '17.0_Z', '18.0_Z', '19.0_Z', '20.0_Z', \
                   'blurring_helpful_1_Z', 'blurring_helpful_2_Z', 'info_sufficient_1_Z', \
                   'info_sufficient_2_Z', 'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z',
                              'irrelevant_distracting_1_Z', 'irrelevant_distracting_2_Z', \
                              'charts_helpful_1_Z', 'charts_helpful_2_Z']])

          ParticipantPublicID    17.0_Z    18.0_Z    19.0_Z    20.0_Z  \
231  5ed57ec8141d24118731f1bc  0.603023  0.603023  0.603023 -2.412091   

     blurring_helpful_1_Z  blurring_helpful_2_Z  info_sufficient_1_Z  \
231             -0.904534              0.603023             0.603023   

     info_sufficient_2_Z  scenarios_helpful_1_Z  scenarios_helpful_2_Z  \
231             0.603023              -0.904534               0.603023   

     irrelevant_distracting_1_Z  irrelevant_distracting_2_Z  \
231                         NaN                         NaN   

     charts_helpful_1_Z  charts_helpful_2_Z  
231                 NaN                 NaN  
232
          ParticipantPublicID    17.0_Z    18.0_Z    19.0_Z    20.0_Z  \
0    5fc94c64c8a3e03ded162dd4  0.000000 -1.224745  0.000000 -1.224745   
1    5f4557043bf01d125f02f01a  0.654654  0.654654  0.654654 -1.527525   
2    5ed7f3404ea9bd2e0ad02df6 -0.095783  0.862044  0.862044 -0.095783   
3    5e4ab1b4ca765d000edebd15  0.862840  0.8

In [245]:
# print(testDF.columns.tolist())
print(testDF.blurring_helpful_1_Z.unique())
print(testDF.blurring_helpful_2_Z.unique())
print(testDF.info_sufficient_1_Z.unique())
print(testDF.info_sufficient_2_Z.unique())
print(testDF.scenarios_helpful_1_Z.unique())
print(testDF.scenarios_helpful_2_Z.unique())
print(testDF.irrelevant_distracting_1_Z.unique())
print(testDF.irrelevant_distracting_2_Z.unique())
print(testDF.charts_helpful_1_Z.unique())
print(testDF.charts_helpful_2_Z.unique())



[ 1.22474487         nan -2.0114352  -1.52656362 -0.53452248  1.03923048
 -1.45174713 -1.06904497 -1.3764944   0.         -2.         -1.22474487
 -0.4472136  -0.77204865  0.5        -0.79240582 -1.83711731  0.18569534
 -1.57798601 -0.84515425  0.65465367 -1.43165827  1.28571429 -2.42487113
 -1.33484762 -0.30151134 -1.69030851 -0.35355339 -1.02513857 -1.22474487
 -1.79231397 -0.30151134 -0.4472136   0.91132238  0.84515425 -1.09321633
  0.69652603 -0.09578263 -1.82574186 -1.35680105  0.10599979 -0.20851441
 -1.19316609  0.91132238 -2.0195672  -1.33978769  0.43643578 -0.5
 -1.06904497 -0.08192319  0.77777778 -1.18321596 -1.11803399  1.76930347
 -1.62221421  0.14285714 -0.58834841 -1.29099445 -1.          1.6583124
 -1.69245584  0.42008403 -1.60591014  0.89442719 -0.82055272 -1.85996222
  0.96225045 -1.62746694  0.5488213  -2.0302589  -0.39223227  0.20412415
 -0.77777778 -0.90535746 -1.62221421 -1.04418513 -0.81649658  0.35355339
 -2.04264872  0.62017367 -1.90443316 -0.54232614 -0.4685212

In [246]:
likertSlim = testDF.copy()

In [247]:
# ## Make DF for Performance on comparisons
# df_CompPivot = tmp_taskDF.query('AgentNum==3 & (Question!=0 | CompConf_Num!=0)')[['Response','Correct','Question', \
#                                                               'ParticipantPublicID', 'CompConf_Num']]
# thisDF = slimDF.copy()
# df_CompPivot['Response'] = pd.Categorical(df_CompPivot.Response)
# print(df_CompPivot.head(4))

# #move confcatvar and compconf up one
# df_CompPivot['Response'] = df_CompPivot['Response'].shift(-1)
# df_CompPivot['CompConf_Num'] = df_CompPivot['CompConf_Num'].shift(-1)
# # df_CompPivot['confCatVar'] = df_CompPivot['confCatVar'].shift(-1)
# df_CompPivot = df_CompPivot.query('Question!=0').dropna().rename(columns={'Question': "likertName"}).drop(columns=['CompConf_Num','Correct'])
# print(df_CompPivot.Response.unique())

# codes = {'Not at All Confident':1,'Not Confident':2, \
#          'Neutral':3,'Somewhat Confident':4,'Completely Confident':5}
# # Now use map on sub-df to swap negative q sentiments
# df_CompPivot['NumericResponse'] = df_CompPivot['Response'].map(codes)
# df_CompPivot.drop('Response', axis=1, inplace=True)
# print(df_CompPivot.head(7))

In [248]:
df_CompPivot = tmp_taskDF.query('AgentNum==3 & (Question!=0 | CompConf_Num!=0)')[['Response','Correct','Question', \
                                                              'ParticipantPublicID']]
thisDF = likertSlim.copy()
df_CompPivot['Response'] = pd.Categorical(df_CompPivot.Response)
print(df_CompPivot.head(4))
# df_pivot.groupby(['VisCond','Question']).plot(kind="bar", title = str("Comparison Confidence"))

#move confcatvar and compconf up one
df_CompPivot['Response'] = df_CompPivot['Response'].shift(-1)
# df_CompPivot['CompConf_Num'] = df_CompPivot['CompConf_Num'].shift(-1)
# df_CompPivot['confCatVar'] = df_CompPivot['confCatVar'].shift(-1)
df_CompPivot = df_CompPivot.dropna()
df_CompPivot = df_CompPivot.query('Question!=0').copy()

# Can we map to numbers here?
codes = {'Not at All Confident':1,'Not Confident':2, 'Neutral':3,'Somewhat Confident':4,'Completely Confident':5}
# Now use map on sub-df to swap negative q sentiments
df_CompPivot['Response'] = df_CompPivot['Response'].map(codes)


print(df_CompPivot.head(4))
tmp2 =  df_CompPivot.pivot(index='ParticipantPublicID', columns=['Question'])
tmp3 = tmp2.stack().reset_index()
print(tmp3.head(4))




tmp_17 = tmp3[tmp3['Question']==17.0]
tmp_17.rename(columns = {'Response': 'Q17_Response', 'Correct': 'Q17_Correct'}, inplace = True)
print(tmp_17.head(4))
thisDF = pd.merge(thisDF,tmp_17[['Q17_Response','Q17_Correct','ParticipantPublicID']],on='ParticipantPublicID',how="left")
print(thisDF.Q17_Response.unique())




tmp_18 = tmp3[tmp3['Question']==18.0]
tmp_18.rename(columns = {'Response': 'Q18_Response', 'Correct': 'Q18_Correct'}, inplace = True)
print(tmp_18.head(4))
thisDF = pd.merge(thisDF,tmp_18[['Q18_Response','Q18_Correct','ParticipantPublicID']],on='ParticipantPublicID',how="left")
print(thisDF.Q18_Response.unique())




tmp_19 = tmp3[tmp3['Question']==19.0]
tmp_19.rename(columns = {'Response': 'Q19_Response', 'Correct': 'Q19_Correct'}, inplace = True)
print(tmp_19.head(4))
thisDF = pd.merge(thisDF,tmp_19[['Q19_Response','Q19_Correct','ParticipantPublicID']],on='ParticipantPublicID',how="left")
print(thisDF.Q19_Response.unique())






tmp_20 = tmp3[tmp3['Question']==20.0]
tmp_20.rename(columns = {'Response': 'Q20_Response', 'Correct': 'Q20_Correct'}, inplace = True)
print(tmp_20.head(4))

print("Added to Slim:")
print(thisDF.head(4))
thisDF = pd.merge(thisDF,tmp_20[['Q20_Response','Q20_Correct','ParticipantPublicID']],on='ParticipantPublicID',how="left")
print(thisDF.Q20_Response.unique())


thisDF['CompCorrectSum'] = thisDF['Q17_Correct'] + thisDF['Q18_Correct'] + thisDF['Q19_Correct'] + thisDF['Q20_Correct']

thisDF['CompResponseMode'] = thisDF[['Q17_Response','Q18_Response','Q19_Response','Q20_Response']].mode(axis=1)[0]
print(thisDF['CompResponseMode'])

likertSlim = pd.merge(likertSlim,thisDF[['Q17_Response','Q17_Correct','Q18_Response','Q18_Correct',
                                 'Q19_Response','Q19_Correct','Q20_Response','Q20_Correct', 'CompCorrectSum',
                                 'CompResponseMode','ParticipantPublicID']],on='ParticipantPublicID',how="left")


                 Response  Correct  Question       ParticipantPublicID
index                                                                 
101        Agent Thompson      0.0      17.0  5fc94c64c8a3e03ded162dd4
102    Somewhat Confident      0.0       0.0  5fc94c64c8a3e03ded162dd4
107         Agent Jackson      1.0      18.0  5fc94c64c8a3e03ded162dd4
108               Neutral      0.0       0.0  5fc94c64c8a3e03ded162dd4
       Response  Correct  Question       ParticipantPublicID
index                                                       
101         4.0      0.0      17.0  5fc94c64c8a3e03ded162dd4
107         3.0      1.0      18.0  5fc94c64c8a3e03ded162dd4
122         4.0      0.0      19.0  5fc94c64c8a3e03ded162dd4
141         3.0      0.0      20.0  5fc94c64c8a3e03ded162dd4
        ParticipantPublicID  Question  Response  Correct
0  55bac673fdf99b554657f2d6      17.0       4.0      1.0
1  55bac673fdf99b554657f2d6      18.0       4.0      1.0
2  55bac673fdf99b554657f2d6      19.0

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [249]:
print(likertSlim.columns.tolist())

['ParticipantPrivateID', 'difficulty', 'visuals', 'agents', 'QsTotal', 'QsCorrect', 'Agent_QsTotal', 'Agent_QsCorrect', 'Agent_QsWO_GhostsTotal', 'Agent_QsWO_GhostsCorrect', 'Qs1_Correct', 'Qs2_Correct', 'QsPart_Total', 'predictionsCorrect', 'predictionsTotal', 'predictions1Correct', 'predictions2Correct', 'predictionsHalfTotal', 'regionsCorrect', 'regionsTotal', 'regions1Correct', 'regions2Correct', 'regionsHalfTotal', 'levelCorrect', 'levelTotal', 'level1Correct', 'level2Correct', 'levelHalfTotal', 'ghostsCorrect', 'ghostsTotal', 'ghosts1Correct', 'ghosts2Correct', 'ghostsHalfTotal', 'comparisonsCorrect', 'comparisonsTotal', 'timeTakenMain_All', 'timeTakenMain_AgentsOnly', 'timeTaken_Consent', 'timeTaken_Demographics', 'timeTaken_AI', 'timeTaken_Agent1', 'timeTaken_Agent2', 'timeTaken_Agent3', 'time_TOTAL', 'vidPlayed', 'avgVidPlays', 'ratioVidPlaysToAvg', 'ExperimentVersion_x', 'totPerc', 'agTotPerc', 'tot1Perc', 'tot2Perc', 'Agent_QsWO_GhostsPerc', 'predPerc', 'pred1Perc', 'pred2Pe

In [250]:
# Get mode of ZSCORES for Likert anwsers on 1, 2, and overall

tmp = likertSlim.copy()

tmp1 = tmp[['blurring_helpful_1_Z', \
      'blurring_helpful_2_Z', 'charts_helpful_1_Z', 'charts_helpful_2_Z', 'info_sufficient_1_Z', \
            'info_sufficient_2_Z', 'irrelevant_distracting_1_Z', 'irrelevant_distracting_2_Z', \
            'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z']]

tmp1['likertMode_Z'] = np.where(tmp1.mode(axis=1)[2].notnull(),tmp1.mode(axis=1)[1],tmp1.mode(axis=1)[0])

print(tmp1['likertMode_Z'])

tmp2 = tmp1[['blurring_helpful_1_Z', 'charts_helpful_1_Z', 'info_sufficient_1_Z', 'irrelevant_distracting_1_Z', \
      'scenarios_helpful_1_Z']]

tmp1['likert1Mode_Z'] = np.where(tmp2.mode(axis=1)[2].notnull(),tmp2.mode(axis=1)[1],tmp2.mode(axis=1)[0])

print(tmp1['likert1Mode_Z'])

tmp2 = tmp1[['blurring_helpful_2_Z', 'charts_helpful_2_Z', 'info_sufficient_2_Z', 'irrelevant_distracting_2_Z', \
      'scenarios_helpful_2_Z']]

tmp1['likert2Mode_Z'] = np.where(tmp2.mode(axis=1)[2].notnull(),tmp2.mode(axis=1)[1],tmp2.mode(axis=1)[0])

print(tmp1['likert2Mode_Z'])

likertSlim['likertMode_Z'] = tmp1['likertMode_Z']
likertSlim['likert1Mode_Z'] = tmp1['likert1Mode_Z']
likertSlim['likert2Mode_Z'] = tmp1['likert2Mode_Z']

tmp1['CompResponseMode_Z'] = likertSlim[['17.0_Z','18.0_Z','19.0_Z','20.0_Z']].mode(axis=1)[0]
likertSlim['CompResponseMode_Z'] = tmp1['CompResponseMode_Z']
print(likertSlim['CompResponseMode_Z'])

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0      1.224745
1      0.654654
2      0.862044
3     -0.730096
4     -1.336306
         ...   
227   -0.171499
228   -0.636364
229    0.000000
230   -0.654654
231    0.603023
Name: likertMode_Z, Length: 232, dtype: float64


/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0      1.224745
1      0.654654
2     -1.053609
3     -0.730096
4     -1.336306
         ...   
227    0.685994
228    0.272727
229    0.000000
230    1.527525
231   -0.904534
Name: likert1Mode_Z, Length: 232, dtype: float64
0      1.224745
1      0.654654
2      0.862044
3     -0.730096
4     -0.534522
         ...   
227   -0.171499
228   -1.545455
229    0.000000
230   -0.654654
231    0.603023
Name: likert2Mode_Z, Length: 232, dtype: float64
0     -1.224745
1      0.654654
2     -0.095783
3      0.862840
4      1.069045
         ...   
227    0.685994
228    1.181818
229    0.000000
230   -0.654654
231    0.603023
Name: CompResponseMode_Z, Length: 232, dtype: float64


/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [251]:
tmp1 = tmp[['blurring_helpful_1_Z', \
      'blurring_helpful_2_Z', 'charts_helpful_1_Z', 'charts_helpful_2_Z', 'info_sufficient_1_Z', \
            'info_sufficient_2_Z', 'irrelevant_distracting_1_Z', 'irrelevant_distracting_2_Z', \
            'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z','17.0_Z','18.0_Z','19.0_Z','20.0_Z']]
tmp1['likertMode_AndComp_Z'] = np.where(tmp1.mode(axis=1)[2].notnull(),tmp1.mode(axis=1)[1],tmp1.mode(axis=1)[0])
likertSlim['likertMode_AndComp_Z'] = tmp1['likertMode_AndComp_Z']

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [252]:
tmp = likertSlim.copy()

tmp1 = tmp[['blurring_helpful_1_Z', \
      'blurring_helpful_2_Z', 'charts_helpful_1_Z', 'charts_helpful_2_Z', 'info_sufficient_1_Z', \
            'info_sufficient_2_Z', 'irrelevant_distracting_1_Z', 'irrelevant_distracting_2_Z', \
            'scenarios_helpful_1_Z', 'scenarios_helpful_2_Z','17.0_Z','18.0_Z','19.0_Z','20.0_Z']]

tmp1['likertMode_All_Z'] = np.where(tmp1.mode(axis=1)[2].notnull(),tmp1.mode(axis=1)[1],tmp1.mode(axis=1)[0])
likertSlim['likertMode_All_Z'] = tmp1['likertMode_All_Z']

/Users/brittdavis/opt/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [253]:
# print(likertSlim['compConf'].unique())
# print(likertSlim['agTotPerc'].unique())
print(likertSlim.columns.tolist())

['ParticipantPrivateID', 'difficulty', 'visuals', 'agents', 'QsTotal', 'QsCorrect', 'Agent_QsTotal', 'Agent_QsCorrect', 'Agent_QsWO_GhostsTotal', 'Agent_QsWO_GhostsCorrect', 'Qs1_Correct', 'Qs2_Correct', 'QsPart_Total', 'predictionsCorrect', 'predictionsTotal', 'predictions1Correct', 'predictions2Correct', 'predictionsHalfTotal', 'regionsCorrect', 'regionsTotal', 'regions1Correct', 'regions2Correct', 'regionsHalfTotal', 'levelCorrect', 'levelTotal', 'level1Correct', 'level2Correct', 'levelHalfTotal', 'ghostsCorrect', 'ghostsTotal', 'ghosts1Correct', 'ghosts2Correct', 'ghostsHalfTotal', 'comparisonsCorrect', 'comparisonsTotal', 'timeTakenMain_All', 'timeTakenMain_AgentsOnly', 'timeTaken_Consent', 'timeTaken_Demographics', 'timeTaken_AI', 'timeTaken_Agent1', 'timeTaken_Agent2', 'timeTaken_Agent3', 'time_TOTAL', 'vidPlayed', 'avgVidPlays', 'ratioVidPlaysToAvg', 'ExperimentVersion_x', 'totPerc', 'agTotPerc', 'tot1Perc', 'tot2Perc', 'Agent_QsWO_GhostsPerc', 'predPerc', 'pred1Perc', 'pred2Pe

In [254]:
# likertSlim['compPerc_Z'] = stats.zscore(likertSlim['compPerc'])
# likertSlim['agTotPerc_Z'] = stats.zscore(likertSlim['agTotPerc'])

In [255]:
# print(likertSlim['compPerc_Z'])
# print(likertSlim['agTotPerc_Z'])

In [257]:
# Write Main Task DF to parquet:
filePath = os.path.join(tempTopDir, 'likertSlim.parquet')
likertSlim.to_parquet(path=filePath,compression='brotli')

# table = pa.Table.from_pandas(likertSlim, safe=False)
# Parquet with Brotli compression
# pq.write_table(table, filePath, compression='BROTLI')

# filePath = os.path.join(tempTopDir, 'GP_Main.csv')
# mainDF.to_csv(filePath)